In [1]:
import numpy as np
import networkx as nx
import sys 
import pickle
from tqdm import tqdm

sys.path.append('../ready functions in py/')
sys.path.append("../../Branching point optimization/code_remastered/")

%load_ext autoreload
%autoreload 2

from general_preprocessing import preprocess_from_topo_to_flows
from iterative_BOT_solver import iterative_bot_solver
from MST_prior import MST_prior_topology
from OT_prior import OT_prior_topology
from angular_stress_heuristic import *
from helper_fcts import *

from mc_update import monte_carlo_step
from incremental_growth import incremental_growth_heuristic
from interpolating_prior import interpolated_prior
from angular_stress_heuristic import *

In [2]:
# idea for every problem devide the cost through the baseline cost.

num_terminals = 100
num_problems = 1
seed = 19
plot_final = True

data_dict = {}
cost_types = np.zeros(2)
np.random.seed(seed)

for i in range(num_problems):

    # generate a random problem:
    num_sources = np.random.randint(1, num_terminals)
    num_sinks = num_terminals - num_sources
    bot_problem_dict = generate_random_bot_problem(num_sources, num_sinks, normalised_to=1, dim=2,
                                                   max_length=1.)
    

    # 1) use the MC baseline:
    N = 50000 #iterations
    tau = 1e-4 #fraction of temperature left after half of the iterations

    al = bot_problem_dict["al"]
    coords_sources = bot_problem_dict["coords_sources"]
    coords_sinks = bot_problem_dict["coords_sinks"]
    supply_arr = bot_problem_dict["supply_arr"]
    demand_arr = bot_problem_dict["demand_arr"]

    # init star graph:
    topo = nx.Graph()
    for node in range(len(supply_arr) + len(demand_arr)):
        topo.add_edge(-1, node)

    cost, coords_iter = iterative_bot_solver(topo, supply_arr, demand_arr, coords_sources, coords_sinks, al,
                                        relative_improvement_threshold=1e-6, min_iterations=-1, max_iterations=1000,
                                        plot=False, title="", fov=None, save=False, save_name="img")

    # MC iterations:
    cost_arr = np.zeros(N)
    T_arr = np.zeros(N)
    #T = cost
    T = 0
    lam = tau**(2/N)
    for i in tqdm(range(N)):
        T = T * lam
        T_arr[i] = T
        topo, cost, coords_iter, accepted = monte_carlo_step(topo, cost, coords_iter, bot_problem_dict, temperature=T)
        cost_arr[i] = cost

    MC_cost, _ = iterative_bot_solver(topo, supply_arr, demand_arr, coords_sources, coords_sinks, al,
                                         relative_improvement_threshold=1e-6, min_iterations=-1, max_iterations=1000,
                                         plot=plot_final, title="", fov=None, save=False, save_name="img")
    MC_topo = topo.copy()

    plt.plot(np.arange(N), cost_arr)
    plt.show()
    plt.plot(np.arange(N), T_arr)
    plt.show()

    # 2) next apply the incremental growth heuristic:
    m = 4
    growth_cost, growth_topo = incremental_growth_heuristic(bot_problem_dict, m, plot=False, final_plot=plot_final)

    # 3) stress reduction with learnable prior:
    beta = 1 - al
    int_prior_topo = interpolated_prior(bot_problem_dict, beta)
    stress_cost, stress_topo = angular_stress_reduction(int_prior_topo, bot_problem_dict, plot_final=plot_final, plot=False)

    cost_types[0] += growth_cost / MC_cost
    cost_types[1] += stress_cost / MC_cost
    
    data_dict[i] = {"bot_problem_dict":bot_problem_dict, 
                    "MC_topo":MC_topo,
                    "growth_topo":growth_topo,
                    "stress_topo":stress_topo,
                    "MC_cost":MC_cost,
                    "growth_cost":growth_cost,
                    "stress_cost":stress_cost
                   }
    
pkl_file_path = f"stored/{num_terminals}_probs{num_problems}_seed{seed}_heuristic_comparison.pkl"
output = open(pkl_file_path, 'wb')
pickle.dump(data_dict, output)
output.close()

  1%|          | 576/50000 [00:35<50:29, 16.31it/s]  


KeyboardInterrupt: 

In [4]:
data_dict

{299: {'bot_problem_dict': {'al': 0.4127429369608906,
   'coords_sources': array([[0.71530938, 0.25787767],
          [0.67563267, 0.04021347],
          [0.11753557, 0.00814826],
          [0.98049457, 0.60324411],
          [0.84901482, 0.54402633],
          [0.05177373, 0.71695237]]),
   'coords_sinks': array([[0.26329489, 0.59745597],
          [0.86711665, 0.46962171],
          [0.18151791, 0.52310778],
          [0.64263515, 0.5593644 ],
          [0.61109228, 0.70377718],
          [0.14339506, 0.97453851],
          [0.33250533, 0.91075544]]),
   'supply_arr': array([0.15669541, 0.19975586, 0.20725158, 0.18590731, 0.02127794,
          0.22911189]),
   'demand_arr': array([0.18297721, 0.15796735, 0.16617097, 0.16206654, 0.09899895,
          0.0971025 , 0.13471648])},
  'MC_topo': <networkx.classes.graph.Graph at 0x7fd67de5fe80>,
  'growth_topo': <networkx.classes.graph.Graph at 0x7fd67dd8d9a0>,
  'stress_topo': <networkx.classes.graph.Graph at 0x7fd67dd7ed00>,
  'MC_cost': 1

In [7]:
growth_cost

1.19554729869105

In [2]:
data_dict = {}
pkl_file_path = f"stored/a.pkl"
output = open(pkl_file_path, 'wb')
pickle.dump(data_dict, output)
output.close()

In [4]:
pkl_file_path

'stored/a.pkl'